### 第五章 变形
#### 一、 长宽表的变形
什么是长表？什么是宽表？这个概念是对于某一个特征而言的。例如：一个表中把性别存储在某一个列中，那么它就是关于性别的长表；如果把性别作为列名，列中的元素是某一其他的相关特征数值，那么这个表是关于性别的宽表。下面的两张表就分别是关于性别的长表和宽表：

In [408]:
import numpy as np 
import pandas as pd

In [409]:
pd.DataFrame({'Gender': ['F','F','M','M'],'Height':[163,160,175,180]})

,Gender,Height
0,F,163
1,F,160
2,M,175
3,M,180


In [410]:
pd.DataFrame({'Height:F':[163,160],'Height:M':[175,180]})

,Height:F,Height:M
0,163,175
1,160,180


显然这两张表从信息上是完全等价的，它们包含相同的身高统计数值，只是这些数值的呈现方式不同，而其呈现方式主要又与性别一列选择的布局模式有关，即到底是以**long**的状态存储还是以**wide**的状态存储。因此，pandas针对此类长宽表的变形操作设计了一些有关的变形函数。

##### 1.pivot
`pivot`是一种典型的长表变宽表的函数，首先看一个例子：下表存储了张三和李四的语文和数学分数，现要把语文和数学分数作为列来展示

In [411]:
df = pd.DataFrame({'Class':[1,1,2,2],
                   'Name':['San Zhang','San Zhang','Si Li','Si Li'],
                   'Subject':['Chinese','Math','Chinese','Math'],
                   'Grade':[80,75,90,85]})
df

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,1,San Zhang,Math,75
2,2,Si Li,Chinese,90
3,2,Si Li,Math,85


对于一个基本的长变宽操作而言，最重要的有三个要素，分别是**变形后的行索引**、**需要转到列索引的列**，**以及这些列和行索引对应的数值**，它们分别对应了`pivot`方法中的`index`, `columns`, `values`参数。新生成表的列索引是`columns`对应列的`unique`值，而新表的行索引是`index`对应列的`unique`值，而`values`对应了想要展示的数值列。

In [412]:
df.pivot(index='Name',columns='Subject',values='Grade')

Subject,Chinese,Math
Name,,
San Zhang,80,75
Si Li,90,85


通过颜色的标记，更容易地能够理解其变形的过程：

利用`pivot`进行变形操作需要满足唯一性的要求，即由于在新表中的行列索引对应了唯一的`value`，因此原表中的`index`和`columns`对应两个列的行组合必须唯一。例如，现在把原表中第二行张三的数学改为语文就会报错，这是由于`Name`与`Subject`的组合中两次出现`("San Zhang", "Chinese")`，从而最后不能够确定到底变形后应该是填写80分还是75分。

In [413]:
df.loc[1,'Subject'] = 'Chinese'
try:
    df.pivot(index='Name',columns='Subject',values='Grade')
except Exception as e:
    Err_Msg = e
Err_Msg

ValueError('Index contains duplicate entries, cannot reshape')

`pandas`从`1.1.0`开始，`pivot`相关的三个参数允许被设置为列表，这也意味着会返回多级索引。这里构造一个相应的例子来说明如何使用：下表中六列分别为班级、姓名、测试类型（期中考试和期末考试）、科目、成绩、排名。

In [414]:
df = pd.DataFrame({'Class':[1,1,2,2,1,1,2,2],
                   'Name':['San Zhang','San Zhang','Si Li','Si Li',
                           'San Zhang','San Zhang','Si Li','Si Li'],
                    'Examination':['Mid','Final','Mid','Final',
                                   'Mid','Final','Mid','Final'],
                    'Subject':['Chinese','Chinese','Chinese','Chinese',
                               'Math','Math','Math','Math'],
                    'Grade':[80,75,85,65,90,85,92,88],
                    'rank':[10,15,21,15,20,7,6,2]})
df

,Class,Name,Examination,Subject,Grade,rank
0,1,San Zhang,Mid,Chinese,80,10
1,1,San Zhang,Final,Chinese,75,15
2,2,Si Li,Mid,Chinese,85,21
3,2,Si Li,Final,Chinese,65,15
4,1,San Zhang,Mid,Math,90,20
5,1,San Zhang,Final,Math,85,7
6,2,Si Li,Mid,Math,92,6
7,2,Si Li,Final,Math,88,2


现在想要把测试类型和科目联合组成的四个类别（期中语文、期末语文、期中数学、期末数学）转到列索引，并且同时统计成绩和排名：

In [415]:
pivot_multi = df.pivot(index=['Class','Name'],columns=['Subject','Examination'],values=['Grade','rank'])
pivot_multi

Grade                     rank                 
Subject         Chinese       Math       Chinese       Math      
Examination         Mid Final  Mid Final     Mid Final  Mid Final
Class Name                                                       
1     San Zhang      80    75   90    85      10    15   20     7
2     Si Li          85    65   92    88      21    15    6     2

根据唯一性原则，新表的行索引等价于对index中的多列使用drop_duplicates，而列索引的长度为values中的元素个数乘以columns的唯一组合数量（与index类似）.

##### 2. pivot_table
`pivot`的使用依赖于唯一性条件，那如果不满足唯一性条件，那么必须通过聚合操作使得相同行列组合对应的多个值变为一个值。例如，张三和李四都参加了两次语文考试和数学考试，按照学院规定，最后的成绩是两次考试分数的平均值，此时就无法通过`pivot`函数来完成。

In [416]:
df = pd.DataFrame({'Name':['San Zhang','San Zhang','Si Li','Si Li','San Zhang','San Zhang','Si Li','Si Li'],
                   'Subject':['Chinese','Chinese','Math','Math','Chinese','Chinese','Math','Math'],
                   'Grade':[80,90,100,90,70,80,85,95]})
df

,Name,Subject,Grade
0,San Zhang,Chinese,80
1,San Zhang,Chinese,90
2,Si Li,Math,100
3,Si Li,Math,90
4,San Zhang,Chinese,70
5,San Zhang,Chinese,80
6,Si Li,Math,85
7,Si Li,Math,95


`pandas`中提供了`pivot_table`来实现，其中的`aggfunc`参数就是使用的聚合函数。上述场景可以如下写出：

In [417]:
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc = 'mean')

Subject,Chinese,Math
Name,,
San Zhang,80.0,NaN
Si Li,NaN,92.5


这里传入`aggfunc`包含了上一章中介绍的所有合法聚合字符串，此外还可以传入以序列为输入标量为输出的聚合函数来实现自定义操作，上述功能可以等价写出：

In [418]:
df.pivot_table(index = 'Name',
               columns = 'Subject',
               values = 'Grade',
               aggfunc = lambda x: x.mean())

Subject,Chinese,Math
Name,,
San Zhang,80.0,NaN
Si Li,NaN,92.5


此外，`pivot_table`具有边际汇总的功能，可以通过设置`margins=True`来实现，其中边际的聚合方式与`aggfunc`中给出的聚合方法一致。下面就分别统计了语文均分和数学均分、张三均分和李四均分，以及总体所有分数的均分：

In [419]:
df.pivot_table(index='Name',columns='Subject',values='Grade',aggfunc='mean',margins=True)

Subject,Chinese,Math,All
Name,,,
San Zhang,80.0,NaN,80.00
Si Li,NaN,92.5,92.50
All,80.0,92.5,86.25


##### 3.melt
长宽表只是数据呈现方式的差异，但其包含的信息量是等价的，前面提到了利用`pivot`把长表转为宽表，那么就可以通过相应的逆操作把宽表转为长表，`melt`函数就起到了这样的作用。在下面的例子中，`Subject`以列索引的形式存储，现在想要将其压缩到一个列中。

In [420]:
df = pd.DataFrame({'Class':[1,2],
                   'Name':['San Zhang', 'Si Li'],
                   'Chinese':[80,90],
                   'Math':[80,75]})
df

,Class,Name,Chinese,Math
0,1,San Zhang,80,80
1,2,Si Li,90,75


In [421]:
df_meltd = df.melt(id_vars = ['Class','Name'], # 保持不变的列（主键/识别列）
                   value_vars = ['Chinese','Math'], # 要被“压缩”的列
                   var_name = 'Subject',  # 被压缩列名的新列名
                   value_name = 'Grade')  # 被压缩值的新列名
df_meltd

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,2,Si Li,Chinese,90
2,1,San Zhang,Math,80
3,2,Si Li,Math,75


参数解释：

- id_vars=['Class','Name']：这些列在 melt 后保持原样，并被复制到每一行。

- value_vars=['Chinese','Math']：这些列将被“拉直”，变成多行。

- var_name='Subject'：原列名（Chinese、Math）将作为值存放在 Subject 列。

- value_name='Grade'：原列中对应的分数存放在 Grade 列。

In [422]:
df_unmeltd = df_meltd.pivot(index= ['Class','Name'],
                            columns = 'Subject',
                            values = 'Grade')
df_unmeltd

,Subject,Chinese,Math
Class,Name,,
1,San Zhang,80,80
2,Si Li,90,75


In [423]:
df_unmeltd = df_unmeltd.reset_index().rename_axis(columns={'Subject': ''})
df_unmeltd.equals(df)

True

##### 4. wide_to_long
`melt`方法中，在列索引中被压缩的一组值对应的列元素只能代表同一层次的含义，即`values_name`。现在如果列中包含了交叉类别，比如期中期末的类别和语文数学的类别，那么想要把`values_name`对应的`Grade`扩充为两列分别对应语文分数和数学分数，只把期中期末的信息压缩，这种需求下就要使用`wide_to_long`函数来完成。

In [424]:
df = pd.DataFrame({'Class':[1,2],
                   'Name':['San Zhang','Si Li'],
                   'Chinese_Mid':[80,75],
                   'Math_Mid':[90,85],
                   'Chinese_Final':[80,75],
                   'Math_Final':[90,85]})
df

,Class,Name,Chinese_Mid,Math_Mid,Chinese_Final,Math_Final
0,1,San Zhang,80,90,80,90
1,2,Si Li,75,85,75,85


In [425]:
pd.wide_to_long(df,
                stubnames = ['Chinese','Math'],   # 它将在所有列中匹配以 Chinese 和 Math 开头的列
                i = ['Class','Name'],     # 长表的唯一标识（主键），每个学生在每场考试会对应一行
                j = 'Examination',   # suffix（如 Mid、Final）将被写入该列
                sep = '_',   #  列名拆分方式：下划线
                suffix = '.+')  #  用正则匹配任意非空字符串作为后缀（Mid、Final 等）

Chinese  Math
Class Name      Examination               
1     San Zhang Mid               80    90
                Final             80    90
2     Si Li     Mid               75    85
                Final             75    85

下面给出一个比较复杂的案例，把之前在`pivot`一节中多列操作的结果（产生了多级索引），利用`wide_to_long`函数，将其转为原来的形态。其中，使用了第八章的`str.split`函数，目前暂时只需将其理解为对序列按照某个分隔符进行拆分即可。

In [426]:
res = pivot_multi.copy()
res.columns = res.columns.map(lambda x : '_'.join(x))
res = res.reset_index()
res = pd.wide_to_long(res, stubnames = ['Grade', 'rank'],
                      i = ['Class','Name'],
                      j = 'Subject_Examination',
                      sep = '_',
                      suffix = '.+')
res

Grade  rank
Class Name      Subject_Examination             
1     San Zhang Chinese_Mid             80    10
                Chinese_Final           75    15
                Math_Mid                90    20
                Math_Final              85     7
2     Si Li     Chinese_Mid             85    21
                Chinese_Final           65    15
                Math_Mid                92     6
                Math_Final              88     2

In [427]:
res = res.reset_index()
res[['Subject','Examination']] = res['Subject_Examination'].str.split('_',expand=True)
res = res.reset_index(drop = True)
res

,Class,Name,Subject_Examination,Grade,rank,Subject,Examination
0,1,San Zhang,Chinese_Mid,80,10,Chinese,Mid
1,1,San Zhang,Chinese_Final,75,15,Chinese,Final
2,1,San Zhang,Math_Mid,90,20,Math,Mid
3,1,San Zhang,Math_Final,85,7,Math,Final
4,2,Si Li,Chinese_Mid,85,21,Chinese,Mid
5,2,Si Li,Chinese_Final,65,15,Chinese,Final
6,2,Si Li,Math_Mid,92,6,Math,Mid
7,2,Si Li,Math_Final,88,2,Math,Final


#### 二、索引的变形
##### 1.stack与unstack
在第二章中提到了利用`swaplevel`或者`reorder_levels`进行索引内部的层交换,下面就要讨论**行列索引之间的交换**,由于这种变换带来了`DataFrame`维度上的变化,因此属于变形操作.        
在第一节中提到的4种变形函数与其不同之处在于，它们都属于某一列或几列**元素**和**列索引**之间的转换，而不是索引之间的转换。       
`unstack`函数的作用是把行索引转换为列索引，例如下面这个简单的例子:

In [428]:
df = pd.DataFrame(np.ones((4,2)),
                  index = pd.Index([('A','cat','big'),
                                    ('A','dog','small'),
                                    ('B','cat','big'),
                                    ('B','dog','small')]),
                columns = ['col_1','col_2'])
df

col_1  col_2
A cat big      1.0    1.0
  dog small    1.0    1.0
B cat big      1.0    1.0
  dog small    1.0    1.0

In [429]:
df.unstack()

col_1       col_2      
        big small   big small
A cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0
B cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0

`unstack`的主要参数是移动的层号，默认转化最内层，移动到列索引的最内层，同时支持同时转化多个层：

In [430]:
df.unstack(2)

col_1       col_2      
        big small   big small
A cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0
B cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0

In [431]:
df.unstack([0,2])

col_1                  col_2                 
        A          B           A          B      
      big small  big small   big small  big small
cat   1.0   NaN  1.0   NaN   1.0   NaN  1.0   NaN
dog   NaN   1.0  NaN   1.0   NaN   1.0  NaN   1.0

类似于`pivot`中的唯一性要求，在`unstack`中必须保证**被转为列索引的行索引层**和**被保留的行索引层**构成的组合是唯一的，例如把前两个列索引改成相同的破坏唯一性，那么就会报错：

In [432]:
my_index = df.index.to_list()
my_index[1] = my_index[0]
df.index = pd.Index(my_index)
df

col_1  col_2
A cat big      1.0    1.0
      big      1.0    1.0
B cat big      1.0    1.0
  dog small    1.0    1.0

In [433]:
try:
    df.unstack()
except Exception as e:
    Err_Msg = e
Err_Msg

ValueError('Index contains duplicate entries, cannot reshape')

与`unstack`相反,`stack`的作用就是把列索引的层压入行索引，其用法完全类似。

In [434]:
df = pd.DataFrame(np.ones((4,2)),
                  index = pd.Index([('A','cat','big'),
                                    ('A','dog','small'),
                                    ('B','cat','big'),
                                    ('B','dog','small')]),
                    columns = ['index_1','index_2']).T
df

A          B      
         cat   dog  cat   dog
         big small  big small
index_1  1.0   1.0  1.0   1.0
index_2  1.0   1.0  1.0   1.0

In [435]:
df.stack()

C:\Users\24153\AppData\Local\Temp\ipykernel_11744\3977694045.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.stack()


A         B     
               cat  dog  cat  dog
index_1 big    1.0  NaN  1.0  NaN
        small  NaN  1.0  NaN  1.0
index_2 big    1.0  NaN  1.0  NaN
        small  NaN  1.0  NaN  1.0

In [436]:
df.stack([1,2])

C:\Users\24153\AppData\Local\Temp\ipykernel_11744\3846085025.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.stack([1,2])


A    B
index_1 cat big    1.0  1.0
        dog small  1.0  1.0
index_2 cat big    1.0  1.0
        dog small  1.0  1.0

##### 2.聚合与变形的关系
在上面介绍的所有函数中，除了带有聚合效果的`pivot_table`以外，所有的函数在变形前后并不会带来`values`个数的改变，只是这些值在呈现的形式上发生了变化。在上一章讨论的分组聚合操作，由于生成了新的行列索引，因此必然也属于某种特殊的变形操作，但由于聚合之后把原来的多个值变为了一个值，因此`values`的个数产生了变化，这也是分组聚合与变形函数的最大区别。

#### 三、其他变形函数
##### 1.crosstab
`crosstab`是一个地位尴尬的函数，因为它能实现的所有功能`pivot_table`都能完成。在默认状态下,`crosstab`可以统计元素组合出现的频数，即`count`操作。例如统计`learn_pandas`数据集中学校和转系情况对应的频数：

In [437]:
df = pd.read_csv('../pandas_basis/learn_pandas.csv')
pd.crosstab(index= df.School, columns= df.Transfer)

Transfer,N,Y
School,,
A,53,0
B,28,2
C,38,1
D,62,4


这等价于如下`crosstab`的如下写法,这里的`aggfunc`即聚合函数:

In [438]:
pd.crosstab(index = df.School, columns = df.Transfer, values = [0]*df.shape[0], aggfunc = 'count')

Transfer,N,Y
School,,
A,53.0,NaN
B,28.0,2.0
C,38.0,1.0
D,62.0,4.0


同样，可以利用`pivot_table`进行等价操作,由于这里统计的是组合的频数,因此`values`参数无论传入哪一个列都不会影响最后的结果：

In [439]:
df.pivot_table(index = 'School', columns = 'Transfer', values = 'Name', aggfunc = 'count')

Transfer,N,Y
School,,
A,53.0,NaN
B,28.0,2.0
C,38.0,1.0
D,62.0,4.0


从上面可以看出这两个函数的区别在于，`crosstab`的对应位置传入的是具体的序列，而`pivot_table`传入的是被调用表对应的名字，若传入序列对应的值则会报错。            
除了默认状态下的`count`统计，所有的聚合字符串和返回标量的自定义函数都是可用的，例如统计对应组合的身高均值：

In [440]:
pd.crosstab(index = df.School, columns = df.Transfer, values = df.Height, aggfunc = 'mean')

Transfer,N,Y
School,,
A,163.953846,NaN
B,163.429630,162.40
C,162.043750,177.20
D,163.253571,164.55


##### 2.explode
`explode`参数能够对某一列的元素进行纵向的展开，被展开的单元格必须存储`list,tuple,Series,np.ndarray`中的一种类型.

In [441]:
df_ex = pd.DataFrame({'A':[[1,2],'my_str',{1,2},pd.Series([3,4])],
                   'B':1})                   
df_ex

,A,B
0,"[1, 2]",1
1,my_str,1
2,"{1, 2}",1
3,0 3 1 4 dtype: int64,1


In [442]:
df_ex.explode('A')

,A,B
0,1,1
0,2,1
1,my_str,1
2,1,1
2,2,1
3,3,1
3,4,1


##### 3.get_dummies
`get_dummies`是用于特征构建的重要函数之一，其作用是把类别特征转为指示变量。例如，对年级一列转为指示变量，属于某一个年级的对应列标记为1，否则为0：

In [443]:
pd.get_dummies(df.Grade).head()

,Freshman,Junior,Senior,Sophomore
0,True,False,False,False
1,True,False,False,False
2,False,False,True,False
3,False,False,False,True
4,False,False,False,True


#### 四、练习
##### Ex1:美国非法药物数据集
现有一份关于美国非法药物的数据集，其中`SubstanceName, DrugReports`分别指药物名称和报告数量:

In [444]:
df = pd.read_csv('drugs.csv')
df.head(3)

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,VA,ACCOMACK,Propoxyphene,1
1,2010,OH,ADAMS,Morphine,9
2,2010,PA,ADAMS,Methadone,2


In [445]:
# 1.
df = df.sort_values(['State','COUNTY','SubstanceName'],ignore_index=True)
res = df.pivot(index = ['State','COUNTY','SubstanceName'], columns = 'YYYY', values = 'DrugReports').reset_index().rename_axis(columns={'YYYY': ''})
res.head(5)

,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,KY,ADAIR,Buprenorphine,NaN,3.0,5.0,4.0,27.0,5.0,7.0,10.0
1,KY,ADAIR,Codeine,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
2,KY,ADAIR,Fentanyl,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,KY,ADAIR,Heroin,NaN,NaN,1.0,2.0,NaN,1.0,NaN,2.0
4,KY,ADAIR,Hydrocodone,6.0,9.0,10.0,10.0,9.0,7.0,11.0,3.0


In [451]:
# 2.
res_melted = res.melt(id_vars = ['State','COUNTY','SubstanceName'],
                     value_vars = res.columns[-8:],
                     var_name = 'YYYY',
                     value_name = 'DrugReports').dropna(subset=['DrugReports'])
res_melted = res_melted[df.columns].sort_values(['State','COUNTY','SubstanceName'],ignore_index=True).astype({'YYYY':'int64','DrugReports':'int64'})
res_melted.equals(df)

True

In [452]:
# 3. 策略一
res = df.pivot_table(index = 'YYYY', columns = 'State', values = 'DrugReports', aggfunc = 'sum')
res.head(3)

State,KY,OH,PA,VA,WV
YYYY,,,,,
2010,10453,19707,19814,8685,2890
2011,10289,20330,19987,6749,3271
2012,10722,23145,19959,7831,3376


In [454]:
# 策略二
pd.crosstab(index = df.YYYY, columns = df.State, values = df.DrugReports, aggfunc = 'sum').head(3)

State,KY,OH,PA,VA,WV
YYYY,,,,,
2010,10453,19707,19814,8685,2890
2011,10289,20330,19987,6749,3271
2012,10722,23145,19959,7831,3376


In [455]:
# 策略三
res = df.groupby(['State','YYYY'])['DrugReports'].sum().to_frame().unstack(0).droplevel(0,axis=1)
res.head(3)

State,KY,OH,PA,VA,WV
YYYY,,,,,
2010,10453,19707,19814,8685,2890
2011,10289,20330,19987,6749,3271
2012,10722,23145,19959,7831,3376


##### Ex2:特殊的wide_to_long方法

In [456]:
df = pd.DataFrame({'Class':[1,2],
                   'Name':['San Zhang','Si Li'],
                   'Chinese':[80,90],
                   'Math':[80,75]})
df

,Class,Name,Chinese,Math
0,1,San Zhang,80,80
1,2,Si Li,90,75


In [461]:
df = df.rename(columns = {'Chinese':'pre_Chinese','Math':'pre_Math'})
# melt方法应当属于wide_to_long的一种特殊情况，即stubnames只有一类
pd.wide_to_long(df,
                stubnames = ['pre'],
                i = ['Class','Name'],
                j = 'Subject',
                sep = '_',
                suffix = '.+').reset_index().rename(columns={'pre':'Grade'})

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,1,San Zhang,Math,80
2,2,Si Li,Chinese,90
3,2,Si Li,Math,75
